In [1]:
from nltk.tokenize import word_tokenize
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from copy import deepcopy
from tqdm import tqdm_notebook as tqdm
import math

C:\programs\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.utils.np_utils import to_categorical
import codecs

In [113]:
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM, Bidirectional, TimeDistributed, GRU
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

Будем генерировать текст на двух произведениях Эдгара По - "Убийство на улице Морг" (тестовое) и "Тайна Мари Роже" (тренировочное). Распределение "тестовая-тренировочная" обусловлено размерами произведений.

In [4]:
text = codecs.open("books/marie.txt", 'r', encoding="utf-8", errors="ignore")
lines = []
for line in text:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    lines.append(line)
text.close()
train = " ".join(lines)

In [5]:
text = codecs.open("books/morgue.txt", 'r', encoding="utf-8", errors="ignore")
lines = []
for line in text:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    lines.append(line)
text.close()
test = " ".join(lines)

In [6]:
train[:1100]

'параллельно с реальными событиями существует идеальная их последовательность. они редко полностью совпадают. люди и обстоятельства обычно изменяют идеальную цепь событий, а потому она кажется несовершенной, и следствия ее равно несовершенны. так было с реформацией – взамен протестантства явилось лютеранство.</ новалис. «взгляд на мораль» * * * примечание: «мари роже» впервые была опубликована без примечаний, поскольку тогда они казались излишними; однако со времени трагедии, которая легла в основу этой истории, прошли годы, а потому появилась нужда и в примечаниях, и в небольшом вступлении, объясняющем суть дела. в окрестностях нью-йорка была убита молодая девушка мэри сесилия роджерс, и хотя это убийство вызвало большое волнение и очень долго оставалось в центре внимания публики, его тайна еще не была раскрыта в тот момент, когда был написан и опубликован настоящий рассказ (в ноябре 1842 г.). автор, якобы описывая судьбу французской гризетки, на самом деле точно и со всеми подробност

In [7]:
test[:1000]

'аналитические способности человека сами по себе весьма мало подходят под анализ. мы ценим их только по их выводами. мы знаем о них только то, что они доставляют человеку громадный источник самых истинных наслаждений. сильный человек наслаждается своей физической мощью, любит упражнения, в которых играют роль его мускулы, а аналитик предпочитает мозговую деятельность, дающую ему возможность исследования. ему доставляют удовольствие даже самые обыкновенные случаи, представляющие возможность применить свои способности, даже загадки, ребусы, иероглифы. способность разгадывания или расследования зависит много от математических знаний, но высшую математику называют несправедливо анализом, потому что не всякий расчет можно назвать этим именем. игрок в шашки, например, очень удачно рассчитывает, не прибегая к анализу. оставляя в стороне абстракции, обратимся к примеру и возьмем игру в шашки, когда действуют только четыре дамки, и, следовательно, нельзя предполагать недостатка внимания. очевид

In [8]:
words_train = [w for w in word_tokenize(train)] # разделим текст пословно
words_test = [w for w in word_tokenize(test)]

In [9]:
unique_train = list(set(words_train)) # посчитаем уникальные слова
unique_test = list(set(words_test))

In [10]:
len(words_train), len(unique_train)

(19218, 4987)

In [11]:
len(words_test), len(unique_test)

(8920, 2781)

In [12]:
words_unique = deepcopy(set(unique_test)) # найдем слова, уникальные для тестового корпуса и удалим их
words_unique.difference_update(set(unique_train))

In [13]:
len(words_unique)

1872

In [14]:
words_test = [w for w in words_test if w not in list(words_unique)] # создадим новый список тестовых слов без уникальных
unique_test = list(set(words_test))

In [15]:
word2index = {w:n for n,w in enumerate(unique_train)} # создадим lookup-словари: из слова в индекс и из индекса в слово на основе слов тренировочной выборки
index2word = {n:w for n,w in enumerate(unique_train)}

In [16]:
WORDS = len(word2index) # длина lookup словаря и будет количеством возможных вариантов предсказаний

In [17]:
SEQLEN = 10 # переведем данные в векторное представление. Так как количество слов в тестовой и тренировочной неодинаково, добавим в тестовую паддинги
STEP = 1 # т.е длина одного варианта-вектора будет одинаковая и в тестовых ответах, и в тренировочных

def to_vectors(words, pad = False):
    global SEQLEN, STEP
    X = []
    Y = []
    for i in range(0, len(words) - SEQLEN, STEP):
        seqs = words[i:i + SEQLEN]
        #print(seqs)
        ans = words[i + SEQLEN]
        X.append([word2index.get(word) for word in seqs])
        Y.append(word2index.get(ans))
    if pad == True:
        return np.array(X),to_categorical(Y, num_classes=WORDS)
    else:
        return np.array(X),to_categorical(Y, num_classes=WORDS)

In [18]:
X_train, Y_train = to_vectors(words_train)
X_test, Y_test = to_vectors(words_test, pad=True)

In [96]:
def perplexity(X_test, model):
    global Y_test, SEQLEN
    probs = []

    for ind in tqdm(range(len(X_test))):
        inp = X_test[ind]
        lab = Y_test[ind]
        lab = list(lab).index(1)
        test_seq = np.reshape(inp, (1, SEQLEN))
        # generating characters
        #x = np.reshape(string_mapped,(1,len(string_mapped)))
        pred = model.predict(test_seq, verbose=0)[0]
        prob = pred[lab]
        probs.append(prob)
    
    log_sum = 0
    for prob in probs:
        log_sum += math.log2(prob)
    mean = -(log_sum/len(probs))
    return math.pow(2, mean)

In [107]:
perplex = {}

In [124]:
def generate(X_test, model): # посмотрим, как модель сможет генерировать текст для 5 рандомных примеров из тестовой выборки
    global WORDS, SEQLEN, word2index, indes2word
    for i in range(5): 
        test_chars = X_test[np.random.randint(len(X_test))]
        test_str = [index2word[i] for i in test_chars] # перекодируем пример обратно в слова
        print("Generating from seed: %s" % (" ".join(test_str)))
        print(*test_str, end=" ")
        for i in range(30): # сгенерируем последовательность из 30 слов за выбранным ядром
            Xtest = np.reshape(test_chars, (1, SEQLEN))
            pred = model.predict_classes(Xtest, verbose=0)
            ypred = [index2word[i] for i in pred] # переводим предсказание из числа в слово
            print(*ypred, end=" ")
            test_chars = np.append(test_chars[1:], pred) # создадим новый входной элемент, включающий в себя уже предсказанное
            #print(*[index2word[i] for i in test_chars])
        print("\n__________")
        print()

Начнем с очень простой модели: один слой LSTM с 64 нейронами и размер эмбеддинга 100

In [19]:
model = Sequential()
model.add(Embedding(WORDS, 100, input_length=SEQLEN))
model.add(LSTM(64))
model.add(Dense(WORDS, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc'])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 100)           498700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 4987)              324155    
Total params: 865,095
Trainable params: 865,095
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_split=0.2, verbose=2)

Train on 15366 samples, validate on 3842 samples
Epoch 1/10
 - 13s - loss: 7.4976 - acc: 0.0957 - val_loss: 7.1800 - val_acc: 0.0953
Epoch 2/10
 - 12s - loss: 6.8002 - acc: 0.0990 - val_loss: 7.3301 - val_acc: 0.0953
Epoch 3/10
 - 11s - loss: 6.7249 - acc: 0.0990 - val_loss: 7.3634 - val_acc: 0.0953
Epoch 4/10
 - 11s - loss: 6.5938 - acc: 0.0990 - val_loss: 7.2870 - val_acc: 0.0953
Epoch 5/10
 - 11s - loss: 6.4447 - acc: 0.0990 - val_loss: 7.3261 - val_acc: 0.0950
Epoch 6/10
 - 12s - loss: 6.3414 - acc: 0.1084 - val_loss: 7.4070 - val_acc: 0.1093
Epoch 7/10
 - 12s - loss: 6.2654 - acc: 0.1118 - val_loss: 7.4531 - val_acc: 0.1111
Epoch 8/10
 - 12s - loss: 6.1940 - acc: 0.1125 - val_loss: 7.4645 - val_acc: 0.1117
Epoch 9/10
 - 12s - loss: 6.1243 - acc: 0.1134 - val_loss: 7.5055 - val_acc: 0.1114
Epoch 10/10
 - 12s - loss: 6.0514 - acc: 0.1138 - val_loss: 7.4838 - val_acc: 0.1111


Точность на обоих выборках мала (хотя валидацию в данной задаче можно отбросить, так как мы не пытаемся выбрать конкретное слово из множества возможных, а пытаемся научить модель строить фразы.

In [108]:
perplex[0] = perplexity(X_test, model)

HBox(children=(IntProgress(value=0, max=6244), HTML(value='')))

Как видно из генерации, данная модель совсем не подходит - она застревает в циклах на одних и тех же словах, так как слишком проста. Попробуем ее усложнить

In [125]:
generate(X_test, model)

Generating from seed: , не ни к какому , на дома . там
, не ни к какому , на дома . там , что в не не не не не не не не не не не не не не не не не не не не не не не не не не не 
__________

Generating from seed: беспорядок в , тело , в вниз , тела ,
беспорядок в , тело , в вниз , тела , что в не не не не не не не не не не не не не не не не не не не не не не не не не не не не 
__________

Generating from seed: их . мы знаем о них только то , что
их . мы знаем о них только то , что в не не не не не не не не не не не не не не не не не не не не не не не не не не не не не 
__________

Generating from seed: , немного , – я вам все , что ;
, немного , – я вам все , что ; в не не не не не не не не не не не не не не не не не не не не не не не не не не не не не 
__________

Generating from seed: . с тем же он отправился к , ее и
. с тем же он отправился к , ее и не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не 
__________



Усложним предыдущую модель:
* Удвоим LSTM и Dropout слои
* Увеличим размер эмбеддингов и неронов в LSTM слоях
* Увеличим число эпох

In [98]:
model3 = Sequential()
model3.add(Embedding(WORDS, 300, input_length=SEQLEN))
model3.add(LSTM(512, return_sequences=True))
model3.add(Dropout(0.2))
model3.add(LSTM(512))
model3.add(Dropout(0.2))
model3.add(Dense(WORDS, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc'])

In [36]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 300)           1496100   
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 512)           1665024   
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 512)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4987)              2558331   
Total params: 7,818,655
Trainable params: 7,818,655
Non-trainable params: 0
_________________________________________________________________


In [37]:
model3.fit(X_train, Y_train, batch_size=128, epochs=30, validation_split=0.2, verbose=2)

Train on 15366 samples, validate on 3842 samples
Epoch 1/30
 - 191s - loss: 7.2004 - acc: 0.0924 - val_loss: 7.1125 - val_acc: 0.0950
Epoch 2/30
 - 165s - loss: 6.7455 - acc: 0.0981 - val_loss: 7.1982 - val_acc: 0.0953
Epoch 3/30
 - 156s - loss: 6.5085 - acc: 0.0990 - val_loss: 7.2380 - val_acc: 0.0953
Epoch 4/30
 - 158s - loss: 6.3386 - acc: 0.1046 - val_loss: 7.3132 - val_acc: 0.1005
Epoch 5/30
 - 157s - loss: 6.1791 - acc: 0.1087 - val_loss: 7.4328 - val_acc: 0.0984
Epoch 6/30
 - 156s - loss: 6.0027 - acc: 0.1100 - val_loss: 7.5142 - val_acc: 0.0940
Epoch 7/30
 - 155s - loss: 5.7992 - acc: 0.1128 - val_loss: 7.5453 - val_acc: 0.0846
Epoch 8/30
 - 155s - loss: 5.5915 - acc: 0.1250 - val_loss: 7.5660 - val_acc: 0.0726
Epoch 9/30
 - 156s - loss: 5.3653 - acc: 0.1341 - val_loss: 7.7364 - val_acc: 0.0796
Epoch 10/30
 - 156s - loss: 5.1263 - acc: 0.1542 - val_loss: 7.8303 - val_acc: 0.0695
Epoch 11/30
 - 155s - loss: 4.9191 - acc: 0.1645 - val_loss: 7.9471 - val_acc: 0.0679
Epoch 12/30
 -

Как мы видим, результаты стали лучше - модель больше не повторяет бесконечно одни и те же слова (хотя все же иногда зацикливается), хотя смысла в последовательностях нет нкакого

In [128]:
generate(X_test, model3)

Generating from seed: и , что дверь , где было найдено тело мадемуазель
и , что дверь , где было найдено тело мадемуазель человеческое человеческое попять попять попять разрушает разрушает разрушает следов следов произвели произвели пропадут пропадут несчастная несчастная лопаток лопаток лопаток лопаток нашлось нашлось предполагаемым труды труды труды труды труды автор автор 
__________

Generating from seed: игрой пустой . действительно , более других заставляет . в
игрой пустой . действительно , более других заставляет . в заметно заметно газетных газетных газетных порождена порождена доказательство доказательство доказательство положения исповеди исповеди восемнадцать восемнадцать восемнадцать гибели людьми дороги дороги глубокой глубокой проверять проверять части части эта-то лицемер травки парфюмера 
__________

Generating from seed: . голосом говорил . несколько слов . , , .
. голосом говорил . несколько слов . , , . справедливым справедливым кости кости выяснилось выяснилось выя

In [109]:
perplex[1] = perplexity(X_test, model3)

HBox(children=(IntProgress(value=0, max=6244), HTML(value='')))

Еще усложним модель той же архитектуры,но сократим число эпох для экономия времени:

In [100]:
model4 = Sequential()
model4.add(Embedding(WORDS, 300, input_length=10))
model4.add(LSTM(700, return_sequences=True))
model4.add(Dropout(0.2))
model4.add(LSTM(700))
model4.add(Dropout(0.2))
model4.add(Dense(WORDS, activation='softmax'))

model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc'])

In [101]:
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 10, 300)           1496100   
_________________________________________________________________
lstm_36 (LSTM)               (None, 10, 700)           2802800   
_________________________________________________________________
dropout_18 (Dropout)         (None, 10, 700)           0         
_________________________________________________________________
lstm_37 (LSTM)               (None, 700)               3922800   
_________________________________________________________________
dropout_19 (Dropout)         (None, 700)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 4987)              3495887   
Total params: 11,717,587
Trainable params: 11,717,587
Non-trainable params: 0
________________________________________________________________

In [103]:
model4.fit(X_train, Y_train, batch_size=128, epochs=15, verbose = 2)

Epoch 1/15
 - 375s - loss: 7.1700 - acc: 0.0950
Epoch 2/15
 - 370s - loss: 6.7283 - acc: 0.0983
Epoch 3/15
 - 349s - loss: 6.5317 - acc: 0.0996
Epoch 4/15
 - 357s - loss: 6.3467 - acc: 0.1068
Epoch 5/15
 - 353s - loss: 6.1281 - acc: 0.1101
Epoch 6/15
 - 367s - loss: 5.8806 - acc: 0.1154
Epoch 7/15
 - 348s - loss: 5.6379 - acc: 0.1272
Epoch 8/15
 - 326s - loss: 5.3906 - acc: 0.1425
Epoch 9/15
 - 344s - loss: 5.1242 - acc: 0.1615
Epoch 10/15
 - 332s - loss: 4.8509 - acc: 0.1776
Epoch 11/15
 - 323s - loss: 4.5690 - acc: 0.1981
Epoch 12/15
 - 355s - loss: 4.2899 - acc: 0.2162
Epoch 13/15
 - 360s - loss: 3.9862 - acc: 0.2439
Epoch 14/15
 - 347s - loss: 3.6891 - acc: 0.2744
Epoch 15/15
 - 332s - loss: 3.3844 - acc: 0.3221


Модель уже лучше генерирует последовательности, меньше повторяет одни и те же слова, по большей части правильно расставляет знаки препинание, а некоторые участки последовательности даже несут смысл: "пока не не успевают извлечь из того , что", "брошенные на реку вскоре после наступления смерти , всплывают , и когда процесс разложения зайдет бы далеко", "что она волочили бы мари", "и сквозь борьба , перевезти на этой сторону". Хотя при этом грамматического согласование почти нет, даже если по смыслу слова сочетаются

In [104]:
generate(X_test, model4)

Generating from seed: . я этого человека к себе с минуты на минуту
. я этого человека к себе с минуты на минуту берег и следует , пока не не успевают извлечь из того , что она действительно вообразить , что преклонный провести часов ткани . затем есть , какие леблан на париже 
__________

Generating from seed: два , и один из них был , несомненно ,
два , и один из них был , несомненно , короткая , но я будем не зайдет бы доказано , что она не мог внимание из дома мари . я , возможно , брошенные на реку , неопровержимо бы не 
__________

Generating from seed: на . он слышал . голосом говорил . несколько слов
на . он слышал . голосом говорил . несколько слов смеркаться , брошенные на реку вскоре после наступления смерти , всплывают , и когда процесс разложения зайдет бы далеко , что она волочили бы мари . для многочисленность не дну 
__________

Generating from seed: до тех пор , пока не ; потом внезапно .
до тех пор , пока не ; потом внезапно . и сквозь борьба , перевезти на этой сторон

In [110]:
perplex[2] = perplexity(X_test, model4)

HBox(children=(IntProgress(value=0, max=6244), HTML(value='')))

Изменим архитектуру: возьмем оптимальные параметры третьей модели (так как они выдали неплохие результаты и при этом не такие тяжелые для ноутбука), но не будем дуплицировать LSTM and Dropout, а также используем вместо простой двунаправленную с активацией релу (вместо tanh по умолчанию)

In [86]:
model5 = Sequential()
model5.add(Embedding(WORDS, 300, input_length=SEQLEN))
model5.add(Bidirectional(LSTM(512, activation = "relu", dropout=0.2, recurrent_dropout=0.2)))
model5.add(Dropout(0.2))
model5.add(Dense(WORDS, activation='softmax'))

model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['categorical_accuracy', 'acc'])

In [87]:
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 10, 300)           1496100   
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 1024)              3330048   
_________________________________________________________________
dropout_15 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 4987)              5111675   
Total params: 9,937,823
Trainable params: 9,937,823
Non-trainable params: 0
_________________________________________________________________


In [88]:
model5.fit(X_train, Y_train, batch_size=128, epochs=50, verbose=2, validation_split=0.2)

Train on 15366 samples, validate on 3842 samples
Epoch 1/50
 - 166s - loss: 7.2845 - categorical_accuracy: 0.0679 - acc: 0.0679 - val_loss: 7.0464 - val_categorical_accuracy: 0.0953 - val_acc: 0.0953
Epoch 2/50
 - 158s - loss: 6.6832 - categorical_accuracy: 0.0990 - acc: 0.0990 - val_loss: 7.0954 - val_categorical_accuracy: 0.0953 - val_acc: 0.0953
Epoch 3/50
 - 158s - loss: 6.4608 - categorical_accuracy: 0.0990 - acc: 0.0990 - val_loss: 7.1758 - val_categorical_accuracy: 0.0958 - val_acc: 0.0958
Epoch 4/50
 - 159s - loss: 6.2459 - categorical_accuracy: 0.1091 - acc: 0.1091 - val_loss: 7.2427 - val_categorical_accuracy: 0.1101 - val_acc: 0.1101
Epoch 5/50
 - 159s - loss: 5.9762 - categorical_accuracy: 0.1154 - acc: 0.1154 - val_loss: 7.3976 - val_categorical_accuracy: 0.1080 - val_acc: 0.1080
Epoch 6/50
 - 157s - loss: 5.6403 - categorical_accuracy: 0.1240 - acc: 0.1240 - val_loss: 7.4659 - val_categorical_accuracy: 0.1104 - val_acc: 0.1104
Epoch 7/50
 - 157s - loss: 5.2153 - categoric

Как мы видим, модель очень быстро набрала высокие показатели точности на тренировочной выборке (хотя и плохо справляется с тестовой). Генерация чуть лучше, чем у третьей модели - все еще хорошая пунктуация, попытка строить осмысленные фразы, но так же в одном контексте начали встречаться семантически связанные слова (пусть и плохо связанные синтаксически): "у кого и рекой, были переправились на берег", "и попросила сент-эсташа зайти за ней, и вечером в пансионе ее", "благодаря идущего известно" (!), "мать и никто не выходили из дома" (!), "в результате разложения" (!)

In [92]:
generate(X_test, model5)

Generating from seed: , показал , что он , в небольшом и иногда
, показал , что он , в небольшом и иногда вещи , чем вытесняет , в большой , у кого и рекой , были переправились на берег берег как их вес нашли , насколько я было возможно , замяли . 
__________

Generating from seed: против другой . к не следует с , хотя непременно
против другой . к не следует с , хотя непременно гораздо более ней , и попросила сент-эсташа зайти за ней , и вечером в пансионе ее , ее ботинки , в связи с тонкими противоречивых человек . три там , 
__________

Generating from seed: время , как свидетели по , убийца не мог .
время , как свидетели по , убийца не мог . благодаря идущего известно , что не более том как будто был других был извлечен довод , мать и никто не выходили из дома . в результате разложения , даже в 
__________

Generating from seed: хотя не дает его . дюпен этим делом , но
хотя не дает его . дюпен этим делом , но они никем представляется и не настолько в то истинности , а в чаще все же 

In [111]:
perplex[3] = perplexity(X_test, model5)

HBox(children=(IntProgress(value=0, max=6244), HTML(value='')))

Оценим также работу ГРУ на тех же данных и с теми же параметрами:

In [114]:
model6 = Sequential()
model6.add(Embedding(WORDS, 300, input_length=10))
model6.add(GRU(512, return_sequences = True))
model6.add(Dropout(0.2))
model6.add(GRU(512))
model6.add(Dropout(0.2))
model6.add(Dense(WORDS, activation='softmax'))

model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['categorical_accuracy', 'acc'])

In [115]:
model6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 10, 300)           1496100   
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 512)           1248768   
_________________________________________________________________
dropout_20 (Dropout)         (None, 10, 512)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 512)               1574400   
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 4987)              2558331   
Total params: 6,877,599
Trainable params: 6,877,599
Non-trainable params: 0
_________________________________________________________________


In [117]:
model6.fit(X_train, Y_train, batch_size=128, epochs=15, verbose=2)

Epoch 1/15
 - 165s - loss: 6.5553 - categorical_accuracy: 0.1088 - acc: 0.1088
Epoch 2/15
 - 165s - loss: 6.1056 - categorical_accuracy: 0.1223 - acc: 0.1223
Epoch 3/15
 - 165s - loss: 5.5398 - categorical_accuracy: 0.1471 - acc: 0.1471
Epoch 4/15
 - 176s - loss: 4.7178 - categorical_accuracy: 0.1955 - acc: 0.1955
Epoch 5/15
 - 170s - loss: 3.7330 - categorical_accuracy: 0.2744 - acc: 0.2744
Epoch 6/15
 - 175s - loss: 2.7326 - categorical_accuracy: 0.4487 - acc: 0.4487
Epoch 7/15
 - 170s - loss: 1.9284 - categorical_accuracy: 0.6033 - acc: 0.6033
Epoch 8/15
 - 163s - loss: 1.3810 - categorical_accuracy: 0.7138 - acc: 0.7138
Epoch 9/15
 - 164s - loss: 1.0129 - categorical_accuracy: 0.7922 - acc: 0.7922
Epoch 10/15
 - 165s - loss: 0.7308 - categorical_accuracy: 0.8566 - acc: 0.8566
Epoch 11/15
 - 166s - loss: 0.5106 - categorical_accuracy: 0.9100 - acc: 0.9100
Epoch 12/15
 - 164s - loss: 0.3668 - categorical_accuracy: 0.9444 - acc: 0.9444
Epoch 13/15
 - 162s - loss: 0.2598 - categorical_

Хотя модель и достигла достаточно высоких показателей точности, предсказывает она все еще неважно, хотя есть последовательности осмысленные, намного лучше,чем предыдущие модели (но они очень похожи на целые куски тестового текста): **"даже если бы в чаще у заставы дюруль в три часа дня , то есть через несколько часов после "**, **"которые по воскресеньям наводняют города"**, **« в трактир ввалилась компания хулиганов , которые вели себя очень буйно , не заплатили за то , что "** (!), **"но полицейские не собиралась начать серьезное расследование , как вдруг в одно прекрасное утро ровно через неделю мари вновь появилась на "**(!), **"успел донести на него"** (!)

In [119]:
generate(X_test, model6)

Generating from seed: что вы заметили ? – я заметил , что все
что вы заметили ? – я заметил , что все какую-то газеты , мадам дюлюк из дома , а все эта свои мари девушки , а во второй случае она не знают никаких , чем под нового . десять шансов 
__________

Generating from seed: все еще о камнях ) до тех пор , пока
все еще о камнях ) до тех пор , пока количество о оторвана примеров , и мысль об наших заметно бы даже если бы в чаще у заставы дюруль в три часа дня , то есть через несколько часов после 
__________

Generating from seed: я не ни одного , ни на минуту не из
я не ни одного , ни на минуту не из драма . связь людей , которые по воскресеньям наводняют города : « в трактир ввалилась компания хулиганов , которые вели себя очень буйно , не заплатили за то , что 
__________

Generating from seed: пор , пока часы не знать , что ночь .
пор , пока часы не знать , что ночь . мы правило , а потому лишь лишь газеты , но полицейские не собиралась начать серьезное расследование , как вдруг

In [120]:
perplex[4] = perplexity(X_test, model6)

HBox(children=(IntProgress(value=0, max=6244), HTML(value='')))

In [132]:
print("One-layer Simple LSTM: ", round(perplex[0],3), "\nBig LSTM: ", round(perplex[1],3), "\nGreat LSTM: ", round(perplex[2],3), "\nBidirectional LSTM: ", round(perplex[3],3), "\nTwo-layered GRU: ", round(perplex[4], 3))

One-layer Simple LSTM:  414.391 
Big LSTM:  4987.051 
Great LSTM:  1696.654 
Bidirectional LSTM:  3949331.39 
Two-layered GRU:  14532.495


Самая низкая перплексия - у самой простой LSTM. Маловероятно, что это отражение качества модели, скорее, того, что она опирается на небольшие параметры, которые не могут учесть всех слов, а значит, у нее меньше вариантов предсказаний, следовательно, она меньше всего в "замешательстве" при выборе последующего слова.  

Наибольшая перплексия (даже очень) - у двунаправленной LSTM. В принципе, это логично, так как двунаправленная ЛСТМ фиксирует контекст и прошлый, и будущий, то есть анализирует больший набор данных. При больших параметрах модели у нее появляется слишком много вариантов возможных слов.

Наиболее оптимальным представляется третий вариант LSTM - у нее не очень высокая перплексия, и хотя отдельные части последовательностей не несут особого смысла в сочетании, отдельные куски вполне осмысленные.

**Что можно еще сделать:**
* так как самым оптимальным оказался вариант, у которого просто были достаточно высокие параметры, можно продолжать их наращивать, чтобы выцепить больше информации из входных данных для модели
* увеличить число эпох, не переходя границу переобучения, чтобы иметь возможность предсказывать новые тексты
* попробовать использовать несколько слоев LSTM
* часто страдает согласование слов, даже при наличии смысловой сочетаемости. Можно было бы помимо непосредственно слов, авать на вход моделям частеречную разметку для более точного предсказания

**Основная проблема** - что делать со словами, уникальными для тестовой выборки. Если их просто игнорировать, не теряем ли мы часть информации контекста, которая может быть важна для моделей LSTM типа, которые именно и ориентированы на запоминание контекста? Если бы была возможность как-то учитывать эти слова, мы могли бы улучшить предсказательную способность модели